# Tanpopo1 表面付着物 VGG16

In [1]:
# Google Colab マウント
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive
import os
os.chdir('/content/drive/MyDrive/Tanpopo')

Mounted at /content/drive
/content/drive/MyDrive


In [2]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import torch.utils.data as data

import matplotlib.pyplot as plt
import glob
import time
import copy
from PIL import Image

plt.ion()

# Google ColabでTensorboardを使うための設定
#from __future__ import absolute_import, division, print_function, unicode_literals
# 日本語訳追記：本セルを追加しないと以下のセルでエラーが発生するため追加
# https://github.com/pytorch/pytorch/issues/30966
#from torch.utils.tensorboard import SummaryWriter
#import tensorflow as tf
#import tensorboard as tb

import torch.nn as nn
import torch.nn.functional as F

# try:
#   # %tensorflow_version only exists in Colab.
#   %tensorflow_version 2.x
# except Exception:
#   print("エラー、やり直してください")
#   pass

#tf.io.gfile = tb.compat.tensorflow_stub.io.gfile

#%load_ext tensorboard

In [3]:
#画像サイズがが704x480 #88x60
img_size = 224

#class_names = ['1Sputter', '2Fiber', '3Block', '4Bar', '5AGFragment']

# 標準化
mean = (0.5, 0.5, 0.5)
std = (0.5, 0.5, 0.5)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#torch.set_default_tensor_type('torch.cuda.FloatTensor')

In [4]:
batch_size =  "16" #@param[8, 16, 32, 64, 128, 256]
batch_size = int(batch_size)

epochs = "22" #@param[5, 8, 10, 15, 20, 22, 25, 27, 29, 30, 31, 32, 33, 35, 45, 60, 120]
epochs = int(epochs)

### 関数、クラスの作成

In [5]:
import random
from sklearn.model_selection import train_test_split
def make_filepath_list(folderpath, phase='train'):
    """
    ファイルのパスを格納したリストを返す
    """
    file_list = []
    test_file_list = []
    train_file_list = []
    valid_file_list = []
    class_names = []

    # .DS_Storeが最初に読み込まれる
    for index, top_dir in enumerate(sorted(os.listdir(folderpath))):
        file_dir = os.path.join(folderpath, top_dir)
        file_list = glob.glob(file_dir + '/*bmp')

        if top_dir != '.DS_Store':
            class_names.append(top_dir)

            if phase == 'test': # テストデータの場合
                test_file_list += [os.path.join(folderpath, top_dir, file).replace('\\', '/') for file in file_list]
                                                            
            else:
                # 各クラス(フォルダ)ごとに8割を訓練データ、2割を検証データとする
                file_list += [os.path.join(folderpath, top_dir, file).replace('\\', '/') for file in file_list]
                train_file, valid_file = train_test_split(file_list, train_size=0.8, test_size=0.2, random_state=0)
                train_file_list += train_file
                valid_file_list += valid_file

    if phase == 'test':
        return test_file_list, class_names

    return train_file_list, valid_file_list, class_names

In [6]:
class ImageTransform(object):
    """
    画像の前処理
    """
    def __init__(self, resize, mean, std):
        self.data_transform = {
            'train': transforms.Compose([
                # データオグメンテーション, 前処理
                transforms.Resize(256), # リサイズ
                transforms.CenterCrop(resize), # 切り取り
                transforms.RandomRotation(45), # ランタムに回転
                transforms.ColorJitter(), # ランダムに明るさ、コントラスト、彩度、色相を変化
                transforms.RandomHorizontalFlip(), #ランダムに左右(水平)反転
                transforms.RandomVerticalFlip(), # ランダムに上下(垂直)反転
                transforms.ToTensor(),
                transforms.Normalize(mean, std), # zcaと交換？
                # ZCA whitening追加する
            ]),
            'valid': transforms.Compose([
                transforms.Resize(256),
                transforms.CenterCrop(resize),
                transforms.ToTensor(),
                transforms.Normalize(mean, std),
            ]),
            'test': transforms.Compose([
                transforms.Resize(256),
                transforms.CenterCrop(resize),
                transforms.ToTensor(),
                transforms.Normalize(mean, std),
            ]),
            'gray': transforms.Compose([
                transforms.Resize(256),
                transforms.CenterCrop(resize),
                transforms.ToTensor(),
                transforms.Normalize(mean, std),
                transforms.Grayscale()
            ])
        }

    def __call__(self, img, phase='train'):
        return self.data_transform[phase](img)

In [7]:
from torchvision.io import read_image

class SurfaceObjectDataset(data.Dataset):
    """
    表面付着物のDatasetクラス
    PyTorchのDatasetクラスを継承
    """
    def __init__(self, file_list, classes, transform=None, phase='train'):
        #super().__init__()
        self.file_list = file_list
        self.transform = transform
        self.classes = classes
        self.phase = phase

        self.img = None
        self.label = None

    def __len__(self):
        """
        画像の枚数を返す
        """
        return len(self.file_list)

    def __getitem__(self, index):
        """
        前処理した画像データのTensor形式のデータとラベルを取得
        """
        # 指定したindexの画像を読み込む
        img_path = self.file_list[index]
        img = Image.open(img_path)

        # 画像ラベルをファイル名から抜き出す
        label = self.file_list[index].split('/')[6][:11]

        # ラベル名を数値に変換
        label = self.classes.index(label)

        # 画像の前処理を実施
        if self.transform is not None:
            img_transformed = self.transform(img, self.phase)
        
        self.img = img_transformed
        self.label = label

        return img_transformed, label

### データ読み込み・前処理

In [8]:
# 訓練、検証データへのファイルパスを格納したリストを取得
train_file_list, valid_file_list, class_names = make_filepath_list('/content/drive/MyDrive/Tanpopo/TrainingData11', 'train')

print('train_file_list: ', train_file_list)
print('class_names: ', class_names)
class_num = len(class_names) # 5

# Datasetの作成
train_dataset = SurfaceObjectDataset(
    file_list = train_file_list, classes = class_names,
    transform = ImageTransform(img_size, mean, std),
    phase = 'train'
)

valid_dataset = SurfaceObjectDataset(
    file_list = valid_file_list, classes = class_names,
    transform = ImageTransform(img_size, mean, std),
    phase = 'valid'
)

# Dataloaderの作成
train_dataloader = data.DataLoader(
    train_dataset, batch_size = batch_size, shuffle=True
)

valid_dataloader = data.DataLoader(
    valid_dataset, batch_size = int(batch_size/2), shuffle=False
)

dataloaders_dict = {
    'train': train_dataloader,
    'valid': valid_dataloader
}

# デフォルトでは `log_dir` としていますが、今回は "runs" とさらに具体的な実験名まで記載します
#writer = SummaryWriter('/content/drive/MyDrive/Tanpopo/runs/surfaceObject_experiment_1')

# # イテレータに変換
# batch_iterator = iter(dataloaders_dict['train'])

# inputs, labels = next(batch_iterator)f

train_file_list:  ['/content/drive/MyDrive/Tanpopo/TrainingData11/1Sputter/SP3A0216S000.bmp', '/content/drive/MyDrive/Tanpopo/TrainingData11/1Sputter/SG3A0055S000.bmp', '/content/drive/MyDrive/Tanpopo/TrainingData11/1Sputter/SF3A0057S010.bmp', '/content/drive/MyDrive/Tanpopo/TrainingData11/1Sputter/SP3A0153S000.bmp', '/content/drive/MyDrive/Tanpopo/TrainingData11/1Sputter/SP3A0254S000.bmp', '/content/drive/MyDrive/Tanpopo/TrainingData11/1Sputter/SP3A0279S011.bmp', '/content/drive/MyDrive/Tanpopo/TrainingData11/1Sputter/SF3A0098S010.bmp', '/content/drive/MyDrive/Tanpopo/TrainingData11/1Sputter/SP3A0279S011.bmp', '/content/drive/MyDrive/Tanpopo/TrainingData11/1Sputter/SM3A0157S008.bmp', '/content/drive/MyDrive/Tanpopo/TrainingData11/1Sputter/SM3A0071S014.bmp', '/content/drive/MyDrive/Tanpopo/TrainingData11/1Sputter/SG3A0024S000.bmp', '/content/drive/MyDrive/Tanpopo/TrainingData11/1Sputter/SM3A0154S009.bmp', '/content/drive/MyDrive/Tanpopo/TrainingData11/1Sputter/SP3A0127S000.bmp', '/cont

### モデルの作成

In [9]:
# モデルをロード
model_vgg16 = {}
model_vgg16 = models.vgg16(pretrained=True) # ImagNetのVGG16学習済みの重みを使用

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

In [10]:
#print(model_vgg16)

In [11]:
# VGG16 の最後の層
model_vgg16.classifier[6] = nn.Linear(in_features=4096, out_features=5)

# 転移学習で学習させるパラメータを、変数params_to_updateに格納
params_to_update = []

# まず全パラメータを勾配計算Falseにする
for name, param in model_vgg16.named_parameters():
    param.requires_grad = False

# 全結合層のみパラメータ更新
for name, param in model_vgg16.classifier.named_parameters():
    param.requires_grad = True
    params_to_update.append(param)

optimizer = optim.AdamW(params_to_update, lr=0.001, weight_decay=0.01)

criterion = nn.CrossEntropyLoss()

In [12]:
!pip install torchinfo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [13]:
#from torchsummary import summary
from torchinfo import summary
model_vgg16.to(device)
print(summary(model=model_vgg16, input_size=(batch_size, 3, 224, 224),
        col_names=['input_size', 'output_size', 'num_params'], 
        device=device,
        #verbose=2
))
print('VGG16')

Layer (type:depth-idx)                   Input Shape               Output Shape              Param #
VGG                                      [16, 3, 224, 224]         [16, 5]                   --
├─Sequential: 1-1                        [16, 3, 224, 224]         [16, 512, 7, 7]           --
│    └─Conv2d: 2-1                       [16, 3, 224, 224]         [16, 64, 224, 224]        (1,792)
│    └─ReLU: 2-2                         [16, 64, 224, 224]        [16, 64, 224, 224]        --
│    └─Conv2d: 2-3                       [16, 64, 224, 224]        [16, 64, 224, 224]        (36,928)
│    └─ReLU: 2-4                         [16, 64, 224, 224]        [16, 64, 224, 224]        --
│    └─MaxPool2d: 2-5                    [16, 64, 224, 224]        [16, 64, 112, 112]        --
│    └─Conv2d: 2-6                       [16, 64, 112, 112]        [16, 128, 112, 112]       (73,856)
│    └─ReLU: 2-7                         [16, 128, 112, 112]       [16, 128, 112, 112]       --
│    └─Conv2d: 2-8

In [ ]:
def train_model(model, criterion, optimizer, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))

        # 各エポックには訓練フェーズと検証フェーズがあります
        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train()  # モデルを訓練モードに設定します
            else:
                model.eval()   # モードを評価するモデルを設定します

            running_loss = 0.0
            running_corrects = 0

            # データをイレテートします
            ##i = 0
            for inputs, labels in dataloaders_dict[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # パラメータの勾配をゼロにします
                optimizer.zero_grad()

                # 順伝播
                # 訓練の時だけ、履歴を保持します
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # 訓練の時だけ逆伝播＋オプティマイズを行います
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # 損失を計算します
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

                # # TensorBoardで可視化
                # if i % 8 == 7: # every 8 mini-batches
                #     if phase == 'train':
                #         writer.add_scalar('training loss', running_loss / 8,
                #                           epoch * len(dataloaders_dict[phase]) + i)
                #         writer.add_figure('Train predictions vs. actuals',
                #             plot_classes_preds(model, inputs, labels),
                #             global_step=epoch * len(dataloaders_dict[phase]) + i)
                #     elif phase == 'valid':
                #         writer.add_scalar('training loss', running_loss / 8,
                #                           epoch * len(dataloaders_dict[phase]) + i)
                #         writer.add_figure('Validation predictions vs. actuals',
                #             plot_classes_preds(model, inputs, labels),
                #             global_step=epoch * len(dataloaders_dict[phase]) + i)
                        
                # i += 1
                        

            # if phase == 'train':
            #     scheduler.step()

            epoch_loss = running_loss / len(dataloaders_dict[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders_dict[phase].dataset)

            print(' {} Loss: {:.4f} Acc: {:.4f} '.format(
                phase, epoch_loss, epoch_acc), end='\t')

            # モデルをディープ・コピーします
            if phase == 'valid' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # ベストモデルの重みをロードします
    model.load_state_dict(best_model_wts)
    return model

### 学習

In [ ]:
PATH = '/content/drive/MyDrive/Tanpopo/model_vgg16_weights.pth'
model_vgg16.load_state_dict(torch.load(PATH)) # 学習前: 前回の重みを使う

<All keys matched successfully>

In [ ]:
# 学習
print('VGG16 Training :')
print('-'*10)
model_vgg16 = model_vgg16.to(device)
model_vgg16 = train_model(model_vgg16, criterion, optimizer, num_epochs=epochs) #Fold-out, Cross Validation
#交差検証では5回ずつ回して、各結果の平均と標準偏差を出す

VGG16 Training :
----------
Epoch 0/21
 train Loss: 3.9915 Acc: 0.3692 	 valid Loss: 0.4938 Acc: 0.8200 	
Epoch 1/21
 train Loss: 0.9016 Acc: 0.6692 	 valid Loss: 0.2032 Acc: 0.9700 	
Epoch 2/21
 train Loss: 0.8047 Acc: 0.7359 	 valid Loss: 0.1914 Acc: 0.9400 	
Epoch 3/21
 train Loss: 0.7900 Acc: 0.7590 	 valid Loss: 0.2041 Acc: 0.9100 	
Epoch 4/21
 train Loss: 0.6322 Acc: 0.8000 	 valid Loss: 0.1951 Acc: 0.9200 	
Epoch 5/21
 train Loss: 0.7821 Acc: 0.7897 	 valid Loss: 0.3149 Acc: 0.8800 	
Epoch 6/21
 train Loss: 0.8225 Acc: 0.7846 	 valid Loss: 0.1497 Acc: 0.9700 	
Epoch 7/21
 train Loss: 0.8926 Acc: 0.7769 	 valid Loss: 0.2368 Acc: 0.9200 	
Epoch 8/21
 train Loss: 0.8741 Acc: 0.7590 	 valid Loss: 0.1785 Acc: 0.9600 	
Epoch 9/21
 train Loss: 0.8705 Acc: 0.7410 	 valid Loss: 0.1421 Acc: 0.9500 	
Epoch 10/21
 train Loss: 0.7046 Acc: 0.8077 	 valid Loss: 0.1281 Acc: 0.9400 	
Epoch 11/21
 train Loss: 0.7886 Acc: 0.7897 	 valid Loss: 0.1373 Acc: 0.9600 	
Epoch 12/21
 train Loss: 0.8633 Ac

In [ ]:
PATH = '/content/drive/MyDrive/Tanpopo/model_vgg16_weights.pth'
#torch.save(model_vgg16.state_dict(), PATH) # 重みを保存

### テスト

In [ ]:
PATH = '/content/drive/MyDrive/Tanpopo/model_vgg16_weights.pth'
#model_vgg16.load_state_dict(torch.load(PATH))

# テストデータ
test_file_list, class_names_test = make_filepath_list('/content/drive/MyDrive/Tanpopo/TestData11', 'test')

print('test_file_list : ', test_file_list)
print('class_names_test : ', class_names_test)

# Datasetの作成
test_dataset = SurfaceObjectDataset(
    file_list = test_file_list, classes = class_names_test,
    transform = ImageTransform(img_size, mean, std),
    phase = 'test'
)

# Dataloaderの作成
test_dataloader = data.DataLoader(
    test_dataset, batch_size = int(batch_size/2), shuffle=False
)

dataloaders_dict['test'] = test_dataloader

<class '__main__.SurfaceObjectDataset'>


In [ ]:
from sklearn.metrics import confusion_matrix
import numpy as np

#正解率
def class_accuracy(label, conf_mat):
    return (conf_mat[label][label] + (np.sum(conf_mat) - (np.sum(conf_mat[:, label])+np.sum(conf_mat[label])-conf_mat[label][label]))) / np.sum(conf_mat)
    
#精度(適合率)
def class_precision(label, conf_mat):
    return conf_mat[label][label] / np.sum(conf_mat[label])

#再現率
def class_recall(label, conf_mat):
    return conf_mat[label][label] / np.sum(conf_mat[:, label])

labels_sum = None
predicted_sum = None

with torch.no_grad():
    for data in dataloaders_dict['test']:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)

        outputs = model_vgg16(images)

        #outputs = nn.Softmax(dim=1)(outputs)

        _, predicted = torch.max(outputs, 1)

        if labels_sum is None:
            labels_sum = labels
            predicted_sum = predicted
        else:
            labels_sum = torch.cat([labels_sum, labels], dim=0)
            predicted_sum = torch.cat([predicted_sum, predicted], dim=0)

In [ ]:
#混同行列
labels_sum = labels_sum.cpu()
predicted_sum = predicted_sum.cpu()
conf_mat = None
print('VGG16 network')
print(class_names_test)

Accuracy = []
Precision = []
Recall = []

conf_mat = confusion_matrix(labels_sum, predicted_sum)
print(conf_mat)
print()
for i in range(class_num):
    Accuracy = np.append(Accuracy, class_accuracy(i, conf_mat)*100)
    Precision = np.append(Precision, class_precision(i, conf_mat)*100)
    Recall = np.append(Recall, class_recall(i, conf_mat)*100)

np.set_printoptions(precision=1)

print('Accuracy : ', Accuracy)
print('Recall : ', Recall)
print('Precision : ', Precision)

VGG16 network
['1Sputter', '2Fiber', '3Block', '4Bar', '5AGFragment']
[[20  1  0  0  0]
 [ 0 21  0  0  0]
 [ 0  0 21  0  0]
 [ 0  0  0 20  1]
 [ 0  2  0  1 18]]

Accuracy :  [ 99.   97.1 100.   98.1  96.2]
Recall :  [100.   87.5 100.   95.2  94.7]
Precision :  [ 95.2 100.  100.   95.2  85.7]


### データと訓練の可視化 (TensorBoard)

In [ ]:
#inputs = inputs.to(device)
#model_vgg16 = model_vgg16.to(device)
##writer.add_graph(model_vgg16, inputs)
##writer.close()
##!tensorboard --logdir=runs

In [ ]:
# # helper function to show an image
# # (used in the `plot_classes_preds` function below)
# def matplotlib_imshow(img, one_channel=False):
#     if one_channel:
#         img = img.mean(dim=0)
#     img = img / 2 + 0.5     # unnormalize
#     npimg = img.to('cpu').numpy()
#     if one_channel:
#         plt.imshow(npimg, cmap="Greys")
#     else:
#         plt.imshow(np.transpose(npimg, (1, 2, 0)))

# # helper function
# #def select_n_random(data, labels, n=500):  # 日本語訳注：500は多いので減らす
# def select_n_random(data, labels, n=100):
#     '''
#     Selects n random datapoints and their corresponding labels from a dataset
#     '''
#     # for img, label in train_dataset:
#     #     print()
#     assert len(data) == len(labels) #assert文はプログラムの内部セルフチェックとして条件をテストするデバッグ支援ツールである

#     perm = torch.randperm(len(data)) #0からn-1までの順列が作成され、ランダムに並び替えられる
#     return data[perm][:n], labels[perm][:n]

# # helper functions

# def images_to_probs(net, images):
#     '''
#     Generates predictions and corresponding probabilities from a trained
#     network and a list of images
#     '''
#     output = net(images)
#     # convert output probabilities to predicted class
#     _, preds_tensor = torch.max(output, 1)
#     preds = np.squeeze(preds_tensor.to('cpu').numpy())
#     return preds, [F.softmax(el, dim=0)[i].item() for i, el in zip(preds, output)]

# def plot_classes_preds(net, images, labels):
#     '''
#     Generates matplotlib Figure using a trained network, along with images
#     and labels from a batch, that shows the network's top prediction along
#     with its probability, alongside the actual label, coloring this
#     information based on whether the prediction was correct or not.
#     Uses the "images_to_probs" function.
#     '''
#     preds, probs = images_to_probs(net, images)
#     # plot the images in the batch, along with predicted and true labels
#     size = images.size()[0] # torch.Size([32, 3, 224, 224])
#     fig = plt.figure(figsize=(12, 48))
#     for idx in np.arange(size):
#         ax = fig.add_subplot(1, size, idx+1, xticks=[], yticks=[])
#         matplotlib_imshow(images[idx], one_channel=True)
#         ax.set_title("{0}, {1:.1f}%\n(label: {2})".format(
#             class_names[preds[idx]],
#             probs[idx] * 100.0,
#             class_names[labels[idx]]),
#                     color=("green" if preds[idx]==labels[idx].item() else "red"))
#     return fig

In [ ]:
# writer.add_graph(model, inputs)
# writer.close()

In [ ]:
# img_sum = None
# label_sum = None
# i=0
# train_dataset_display = SurfaceObjectDataset(
#     file_list = train_file_list, classes = class_names,
#     transform = ImageTransform(img_size, mean, std),
#     phase = 'gray' # To.Tensorを定期用したいが,チャネル数3より、5748ではなく1916のまま
# )
# for input, label in train_dataset_display:
#                 input = input.to(device)
#                 #label = label.to(device)
                
#                 if img_sum is None:
#                     img_sum = input
#                     label_sum = label
                    
#                 #img_sum = np.append(img_sum, input)
#                 else:
#                     img_sum = torch.cat([img_sum, input], dim=0)
                
#                     label_sum = np.append(label_sum, label)

#                 #label_sum = torch.cat([label_sum, label], dim=0)

#                 i += 1

In [ ]:
# #print(img_sum)
# print(len(img_sum))
# print(img_sum.size())
# print('i : ', i)
# print(label_sum)
# print(len(label_sum))

In [ ]:
# # select random images and their target indices
# #img, lab = dataloaders_dict['train']
# images, labels = select_n_random(img_sum, label_sum)

# # get the class labels for each image
# class_labels = [class_names[lab] for lab in labels]

# # log embeddings
# features = images.view(-1, img_size * img_size)
# writer.add_embedding(features,
#                     metadata=class_labels,
#                     label_img=images.unsqueeze(1))
# writer.close()

In [ ]:
#%tensorboard --logdir /content/drive/MyDrive/Tanpopo/runs/surfaceObject_experiment_1/